In [1]:
##### Imports ######
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import json
import re

In [67]:
# Removes XML tags from the data
def clean_data(data):
    try:
        tags = ET.fromstring(str(data))
        return ET.tostring(tags, method='text').decode("utf-8").replace('\'', '').replace('\ ', '').replace('\"','')
    except:
        return ''

# Finds all different places and removes the XML tags from them
def clean_places(places):
    placelist = []
    for i in places:
        tags = ET.fromstring(str(i))
        placelist.append(ET.tostring(tags, method='text').decode("utf-8"))
        
    return placelist

# Removes title, place and the XML tags from the text
def clean_text(text):
    for tag in text.findAll():
        tag.replaceWith('')
    
    text = text.text.replace('\n',' ').replace('\"', '').replace('\'', '').replace('\ ','')
    
    return re.sub('[\x7f]', '', text)

def clean_year(data):
    year = re.search('[0-9]+-[A-Z]+', str(data)).group(0)
    return str(year)

# Creates dictionary with all documents in the Reuters database, assigns doc_ID to all documents. 
def load_data():
    data_dict = {}
    data_id = 0
    
    with open('reuters.json', 'w') as f:
        for i in range(22):
            if i < 10:
                data1 = open('reuters_data/reut2-00'+ str(i) +'.sgm')
            else:
                data1 = open('reuters_data/reut2-0'+ str(i) +'.sgm')
            soup = BeautifulSoup(data1,'lxml')
            items = soup.findAll('reuters')

            for doc in items:
                ID =     {'index':{'_id':str(data_id)}}
                data =   {'date':clean_year(doc.date),
                          'topic':clean_data(doc.topics), 
                          'place':clean_places(doc.places.findAll('d')), 
                          'people':clean_data(doc.people), 
                          'orgs':clean_data(doc.orgs), 
                          'exchanges':clean_data(doc.exchanges), 
                          'companies':clean_data(doc.companies), 
                          'title':clean_data(doc.title), 
                          'text':clean_text(doc.findAll('text')[0]) 
                          }
                
                json.dump(ID, f)
                f.write('\n')
                json.dump(data, f)
                f.write('\n')

                print('Indexing document', data_id + 1, 'of 21578', end='\r')
                data_id += 1
        
    return 'done'

In [68]:
load_data()

'done'